In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

In [2]:
train = pd.read_csv("D:/kaggle_data/santander-product-recommendation/train_ver2.csv")
test = pd.read_csv("D:/kaggle_data/santander-product-recommendation/test_ver2.csv")
sample_submission = pd.read_csv("D:/kaggle_data/santander-product-recommendation/sample_submission.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   fecha_dato             object 
 1   ncodpers               int64  
 2   ind_empleado           object 
 3   pais_residencia        object 
 4   sexo                   object 
 5   age                    object 
 6   fecha_alta             object 
 7   ind_nuevo              float64
 8   antiguedad             object 
 9   indrel                 float64
 10  ult_fec_cli_1t         object 
 11  indrel_1mes            object 
 12  tiprel_1mes            object 
 13  indresi                object 
 14  indext                 object 
 15  conyuemp               object 
 16  canal_entrada          object 
 17  indfall                object 
 18  tipodom                float64
 19  cod_prov               float64
 20  nomprov                object 
 21  ind_actividad_cliente  float64
 22  renta           

In [4]:
#상품을 제외한 데이터 제외
train=train.drop(['fecha_dato','ind_empleado','pais_residencia','sexo','age','fecha_alta','ind_nuevo','antiguedad','indrel',\
                        'ult_fec_cli_1t','indrel_1mes','tiprel_1mes','indresi','indext','conyuemp','canal_entrada',\
                       'indfall','tipodom','cod_prov','nomprov','ind_actividad_cliente','renta','segmento'],axis=1)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 25 columns):
 #   Column             Dtype  
---  ------             -----  
 0   ncodpers           int64  
 1   ind_ahor_fin_ult1  int64  
 2   ind_aval_fin_ult1  int64  
 3   ind_cco_fin_ult1   int64  
 4   ind_cder_fin_ult1  int64  
 5   ind_cno_fin_ult1   int64  
 6   ind_ctju_fin_ult1  int64  
 7   ind_ctma_fin_ult1  int64  
 8   ind_ctop_fin_ult1  int64  
 9   ind_ctpp_fin_ult1  int64  
 10  ind_deco_fin_ult1  int64  
 11  ind_deme_fin_ult1  int64  
 12  ind_dela_fin_ult1  int64  
 13  ind_ecue_fin_ult1  int64  
 14  ind_fond_fin_ult1  int64  
 15  ind_hip_fin_ult1   int64  
 16  ind_plan_fin_ult1  int64  
 17  ind_pres_fin_ult1  int64  
 18  ind_reca_fin_ult1  int64  
 19  ind_tjcr_fin_ult1  int64  
 20  ind_valo_fin_ult1  int64  
 21  ind_viv_fin_ult1   int64  
 22  ind_nomina_ult1    float64
 23  ind_nom_pens_ult1  float64
 24  ind_recibo_ult1    int64  
dtypes: float64(2), i

In [6]:
train.head()

,ncodpers,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,1375586,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,1050611,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,1050612,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,1050613,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,1050614,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [7]:
train.duplicated()

0           False
1           False
2           False
3           False
4           False
            ...  
13647304     True
13647305     True
13647306     True
13647307     True
13647308    False
Length: 13647309, dtype: bool

In [8]:
train = train.drop_duplicates(['ncodpers'], keep='last') #중복값 제거

In [9]:
train.head()

,ncodpers,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
1922,1049700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,0
2989,1055504,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,0
5666,1045269,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,0
6196,1042585,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,0
7440,1047273,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,0


In [10]:
train=train.fillna(0) #결측치 처리

In [11]:
train.head()

,ncodpers,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
1922,1049700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
2989,1055504,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
5666,1045269,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
6196,1042585,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
7440,1047273,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [12]:
models = {}
model_preds = {}
id_preds = defaultdict(list)
ids = train['ncodpers'].values

In [13]:
#각 상품마다 logistic regression
for c in train.columns:
    if c != 'ncodpers':
        y_train = train[c]
        print('c='+str(c))
        print('y_train='+str(train[c]))
        
        x_train = train.drop([c, 'ncodpers'], 1)
        
        clf = LogisticRegression()
        clf.fit(x_train, y_train)
        p_train = clf.predict_proba(x_train)[:,1]
        
        models[c] = clf
        model_preds[c] = p_train
        for id, p in zip(ids, p_train):
            id_preds[id].append(p)
            

c=ind_ahor_fin_ult1
y_train=1922        0
2989        0
5666        0
6196        0
7440        0
           ..
13647304    0
13647305    0
13647306    0
13647307    0
13647308    0
Name: ind_ahor_fin_ult1, Length: 956645, dtype: int64


C:\Users\Min\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


c=ind_aval_fin_ult1
y_train=1922        0
2989        0
5666        0
6196        0
7440        0
           ..
13647304    0
13647305    0
13647306    0
13647307    0
13647308    0
Name: ind_aval_fin_ult1, Length: 956645, dtype: int64


C:\Users\Min\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


c=ind_cco_fin_ult1
y_train=1922        0
2989        0
5666        0
6196        0
7440        0
           ..
13647304    1
13647305    1
13647306    1
13647307    1
13647308    1
Name: ind_cco_fin_ult1, Length: 956645, dtype: int64


...

C:\Users\Min\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


c=ind_nom_pens_ult1
y_train=1922        0.0
2989        0.0
5666        0.0
6196        0.0
7440        0.0
           ... 
13647304    0.0
13647305    0.0
13647306    0.0
13647307    0.0
13647308    0.0
Name: ind_nom_pens_ult1, Length: 956645, dtype: float64


C:\Users\Min\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


c=ind_recibo_ult1
y_train=1922        0
2989        0
5666        0
6196        0
7440        0
           ..
13647304    0
13647305    0
13647306    0
13647307    0
13647308    0
Name: ind_recibo_ult1, Length: 956645, dtype: int64


C:\Users\Min\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [14]:
#이미 있는 상품을 제외
already_active = {}
for row in train.values:
    row = list(row)
    id = row.pop(0)
    active = [c[0] for c in zip(train.columns[1:], row) if c[1] > 0]
    already_active[id] = active

In [15]:
#상품 7개 추천
train_preds = {}
for id, p in id_preds.items():
    preds = [i[0] for i in sorted([i for i in zip(train.columns[1:], p) if i[0] not in already_active[id]], key=lambda i:i [1], reverse=True)[:8]]
    train_preds[id] = preds

In [16]:
test_preds = []
for row in sample_submission.values:
    id = row[0]
    p = train_preds[id]
    test_preds.append(' '.join(p))

In [17]:
sample_submission['added_products'] = test_preds
sample_submission.to_csv('submission.csv', index=False)